A notebook to package one set of statistical results into a collection

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

from keller_zlatic_vnc.collections import form_collection

## Parameters go here

In [3]:
# Folder containing produced maps
image_folder = r'\\dm11\bishoplab\projects\keller_vnc\results\initial_single_subject_small_window_sweep\ind_collections\beh_stats_neg_3_3_turns_broken_out_mean_cmp_stats_images'

# Give path to a file holding parameters used to extract fluorescece
f_extraction_params_file = r'K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_1_5_5\extraction_params.pkl'

# Give path to a file holding parameters used to extract baselines
baseline_calc_params_file = r'K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_1_5_5\long_baseline_extract_params.pkl'

# Give path to a file holding parameters for the initial model fitting
mdl_fitting_params_file = r'\\dm11\bishoplab\projects\keller_vnc\results\initial_single_subject_small_window_sweep\ind_collections\beh_stats_neg_3_3_turns_broken_out.pkl'

# Give path to a file holding parameters for calculating statistics when comparing means
mean_cmp_params_file = r'\\dm11\bishoplab\projects\keller_vnc\results\initial_single_subject_small_window_sweep\ind_collections\beh_stats_neg_3_3_turns_broken_out_mean_cmp_stats.pkl'

# Path to folder collection should be created in
tgt_folder = image_folder

# List those who can be contacted with questions about the collection
responsible = ['William Bishop <bishopw@hhmi.janelia.org>', 'Andrew Champion <andrew.champion@gmail.com>']

# Provide a description of the collection. 
description = ('This is part of an initial analysis performed to generate a set of maps that can serve as a testbed for ' + 
               'integrating statistical maps with EM. This particular analysis contains results for a statistical analysis ' + 
               'of the single specimen for which we also have an EM volumen for, and we look for voxels which encode an ' + 
               'upcoming behavior.  This is only one analysis in a larger analysis in which we vary the location of the window ' + 
               'we analyze neural activity in.  Note that because the actual statistical results were performed before we ' + 
               'determined how we were going to record metadata for the collections, the GIT hashes in this metadata ' + 
               'are only approximate, as the exact hashes were not recorded at the time the statistical testing was done. ' + 
               'The key notebooks and scripts to run the statistical tests used to produce these maps are ' + 
               'spont_events_initial_stats_calculation.ipynb and find_vls_different_than_other_mean.py.  The script ' + 
               'make_spont_whole_brain_movies_and_images.py was then used to render the actual maps.')

# List hashes identify commits in git for the different pieces of code used to produce these results
git_hashes = {'janelia_core': '004754091f3e793b681987d72341185826f5de79', 
             'keller_zlatic_vnc': 'beda4ab71553e0a3693af0f37c853f5d2966fee2'}

# List the parameters that should be included in the metadata file
f_extraction_yaml_fields = {'voxel_size_per_dim': 'Number of voxels in each dimension of a supervoxel.'}

baseline_calc_yaml_fields = {'window_length': 'Length of window used for baseline calculation.',
                             'filter_start': 'Initial offset, relative to first data point, of window used for baseline calculations.', 
                             'write_offset': "Offset between first point in window and the point the filtered output is assigned to.", 
                             'p': 'The particular percentile used for percentile filtering.'}

mdl_fitting_yaml_fields = {'q_th': 'The threshold in times points used for determining if a behavior is preceeded by quiet.', 
                           'enforce_contained_events': 'True if only events with behaviors falling entirely within the window of analyzed neural activity should be included in the data used for model fitting.',
                           'pool_preceeding_behaviors': 'True if all preceeding behaviors should be pooled.', 
                           'pool_preceeding_turns': 'True if left and right preceeding turns should be pooled.',
                           'pool_succeeding_turns': 'True if succeeding left and right turns should be pooled.',
                           'clean_event_def': 'The criteria used for determing which events are clean, with respect to how they overlap other events, for inclusion in the analysis.',
                           'behaviors': 'Filter applied so that only behaviors in this set are included in the analysis.',
                           'min_n_subjs': 'The min number of subjects we need to observe a transition for in order to include it in the model.',
                           'min_n_events': 'The min number of times we need to observe a transition (across all subejcts) to include it in the model.',
                           'window_type': 'The method for determining the alignment of the window of neural activity relative to behavior.',
                           'window_offset': 'The offset in time points of the start of the window of neural activity relative to behaivor.',
                           'window_length': 'The length of the window in time points of neural activity analyzed.'
                           }

mean_cmp_yaml_fields = {}


## Load the pickle files containing parameters

In [4]:
with open(f_extraction_params_file, 'rb') as f:
    f_extraction_params = pickle.load(f)

In [5]:
with open(baseline_calc_params_file, 'rb') as f:
    baseline_calc_params = pickle.load(f)

In [6]:
with open(mdl_fitting_params_file, 'rb') as f:
    mdl_fitting_results = pickle.load(f)

In [7]:
with open(mean_cmp_params_file, 'rb') as f:
    mean_cmp_results = pickle.load(f)

## Determine the set of preceeding and succeeding behaviors

In [8]:
preceding_behs = list(set(t[0] for t in mean_cmp_results['beh_trans']))
preceding_behs.sort()
suceeding_behs = list(set(t[1] for t in mean_cmp_results['beh_trans']))
suceeding_behs.sort()

## Form the collection

In [9]:
params = [{'desc': 'f_extraction_params', 
          'for_saving': f_extraction_params, 
          'for_metadata': f_extraction_params['roi_extract_opts'], 
          'inc_params': f_extraction_yaml_fields}, 
          {'desc': 'baseline_calc_params', 
          'for_saving': baseline_calc_params, 
          'for_metadata': baseline_calc_params['baseline_calc_opts'], 
          'inc_params': baseline_calc_yaml_fields},
          {'desc': 'mdl_fitting_params', 
           'for_saving': mdl_fitting_results['ps'], 
           'for_metadata': mdl_fitting_results['ps'], 
           'inc_params': mdl_fitting_yaml_fields},
          {'desc': 'mean_cmp_params', 
           'for_saving': mean_cmp_results['ps'], 
           'for_metadata': mean_cmp_results['ps'], 
           'inc_params': mean_cmp_yaml_fields}]

In [10]:
form_collection(image_folder=image_folder, 
                tgt_folder=tgt_folder,
                description=description,
                responsible=responsible,
                git_hashes=git_hashes,
                preceding_behs=preceding_behs,
                suceeding_behs=suceeding_behs,
                params=params, 
                ignore_extensions=['.png'])